In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
import logging
import sys


In [2]:
sys.path.append(os.path.join(os.path.abspath('..')))
# Import modules
from src import data_loader as dl
from scripts.Feature_Engineering import create_aggregate_features,extract_time_features,encode_categorical_variables,check_and_handle_missing_values,standardize_numerical_features,construct_rfms_scores,save_transformed_data

2025-03-07 16:27:04,375 - INFO - Directory already exists: c:\Users\HP\Documents\Dagii\Credit-scoring-model\data
2025-03-07 16:27:04,377 - INFO - Directory already exists: c:\Users\HP\Documents\Dagii\Credit-scoring-model\src\../logs
